In [17]:
import numpy as np

# np.zeros((5,4))
np.random.randint(1,7,(5,4))

array([[4, 6, 3, 4],
       [6, 1, 6, 5],
       [1, 3, 5, 1],
       [4, 3, 4, 2],
       [3, 1, 3, 2]])

In [4]:
from collections import Counter

c = Counter([1,2,3])
c[4]

0

In [20]:
import sys
sys.maxsize

-9223372036854775807

## Cloud

In [ ]:
#Read many part files from AWS s3

import boto3, io, s3fs
from multiprocessing import Pool

def serial_read_part_files_to_df(bucket_name, parent_key):
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(bucket_name)
    prefix_objs = bucket.objects.filter(Prefix=parent_key)
    sub_df = []
    for obj in prefix_objs:
        file_key = obj.key
        print(file_key)
        cur_df = pd.read_csv('s3://'+bucket_name+'/'+file_key, compression='gzip', header=0, sep='|', quotechar='"')
        cur_df = cur_df[valid_col]
        sub_df.append(cur_df)
    return pd.concat(sub_df)

def parallel_read_part_files_to_df(bucket_name, parent_key):
    def get_data(file_key):
        print(file_key)
        cur_df = pd.read_csv('s3://'+bucket_name+'/'+file_key, compression='gzip', header=0, sep='|', quotechar='"')
        cur_df = cur_df[valid_col]
        sub_df.append(cur_df)        
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(bucket_name)
    prefix_objs = bucket.objects.filter(Prefix=parent_key)
    file_keys = tuple(map(lambda x : x.key, prefix_objs))
    sub_df = []
    agents, chunksize = 4, 5
    with Pool(processes=agents) as pool:
        sub_df = pool.map(get_data, file_keys, chunksize)
    return pd.concat(sub_df)
